In [1]:
import pandas as pd
import matplotlib.pyplot as pp
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn import tree
from sklearn.ensemble import AdaBoostClassifier as ab
%matplotlib inline

In [13]:
df = pd.read_csv('train.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [14]:
cdf = df.copy()
cdf = cdf.drop(['Ticket', 'Name', 'Cabin', 'PassengerId', 'Embarked'], axis=1)
cdf.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,0,3,male,22.0,1,0,7.2500
1,1,1,female,38.0,1,0,71.2833
2,1,3,female,26.0,0,0,7.9250
3,1,1,female,35.0,1,0,53.1000
4,0,3,male,35.0,0,0,8.0500


In [15]:
cdf.shape

(891, 7)

In [28]:
def ageGroup(age):
#     if age <= 2: return 0
#     if age <= 5: return 3
#     if age <= 10: return 7
#     if age <= 15: return 13
    if age <= 100: return age
    else: return 29.699118

In [8]:
cdf['FarePH'] = cdf['Fare'] / (cdf['SibSp'] + cdf['Parch'] + 1)

cdf['Age'] = cdf['Age'].apply(lambda age: ageGroup(age)).astype('category')
# cdf['Child'] = np.where(cdf['Age'] < 16, 1, 0)
# cdf = cdf.drop('Age', axis=1)


In [16]:
cdf = cdf.dropna(axis=0, how='any')
cdf.shape

(714, 7)

In [10]:
cdf[cdf['Age'].isnull()]

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare


In [17]:
cdf['Sex'] = cdf['Sex'].apply({'male':0, 'female':1}.get).astype('category')
cdf.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,0,3,0,22.0,1,0,7.2500
1,1,1,1,38.0,1,0,71.2833
2,1,3,1,26.0,0,0,7.9250
3,1,1,1,35.0,1,0,53.1000
4,0,3,0,35.0,0,0,8.0500


In [31]:
#cdf['Embarked'] = cdf['Embarked'].apply({'C':1, 'Q':2, 'S':3, None:4}.get).astype('category')
cdf.sort_values(by='FarePH', ascending=False)[:5]

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,FarePH
737,1,1,0,35.0,0,0,512.3292,512.3292
258,1,1,1,35.0,0,0,512.3292,512.3292
679,1,1,0,36.0,0,1,512.3292,256.1646
380,1,1,1,42.0,0,0,227.5250,227.5250
716,1,1,1,38.0,0,0,227.5250,227.5250


In [19]:
cdf_age_known =  cdf[cdf['Age'].isnull() == False]
cdf_age_unknown = cdf[cdf['Age'].isnull()]

In [20]:
# cdf_age_known_X, cdf_age_known_y = (cdf_age_known.drop(['Survived', 'FarePH'], axis=1)), cdf_age_known['Survived']
cdf_age_known_X, cdf_age_known_y = (cdf_age_known.drop(['Survived'], axis=1)), cdf_age_known['Survived']


In [21]:
 cdf_age_known_X_train, cdf_age_known_X_test, cdf_age_known_y_train, cdf_age_known_y_test \
    = train_test_split(cdf_age_known_X, cdf_age_known_y, test_size=0.2)

In [22]:
tree_clf = ab(n_estimators=500, learning_rate=0.3)

In [23]:
# tree_clf = tree.DecisionTreeClassifier()


In [24]:
tree_clf.fit(cdf_age_known_X_train, cdf_age_known_y_train)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=0.3, n_estimators=500, random_state=None)

In [25]:
tree_clf.score(cdf_age_known_X_test, cdf_age_known_y_test)

0.80419580419580416

In [26]:
tree_clf.score(cdf_age_known_X_train, cdf_age_known_y_train)

0.8528896672504378

In [27]:
len(cdf_age_unknown)

0

In [15]:
tdf = pd.read_csv('test.csv')
tdf['Fare'] = tdf['Fare'].fillna(cdf[cdf['Pclass'] == 3]['FarePH'].mean())

In [16]:
tdf[tdf['PassengerId'] == 1044 ]

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
152,1044,3,"Storey, Mr. Thomas",male,60.5,0,0,3701,8.094756,NaN,S


In [17]:
# tdf['FarePH'] = tdf['Fare'] / (tdf['SibSp'] + tdf['Parch'] + 1)
tdf = tdf.drop(['Ticket', 'Name', 'Cabin', 'Embarked'], axis=1)
tdf['Age'] = tdf['Age'].apply(lambda age: ageGroup(age)).astype('category')
len(tdf)

418

In [18]:
cdf.dtypes

Survived       int64
Pclass         int64
Sex         category
Age         category
SibSp          int64
Parch          int64
Fare         float64
FarePH       float64
dtype: object

In [19]:
tdf.dtypes

PassengerId       int64
Pclass            int64
Sex              object
Age            category
SibSp             int64
Parch             int64
Fare            float64
dtype: object

In [20]:
tdf['Sex'] = tdf['Sex'].apply({'male':0, 'female':1}.get).astype('category')
#cdf['Embarked'] = cdf['Embarked'].apply({'C':1, 'Q':2, 'S':3, None:4}.get).astype('category')
tdf.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare
0,892,3,0,34.5,0,0,7.8292
1,893,3,1,47.0,1,0,7.0000
2,894,2,0,62.0,0,0,9.6875
3,895,3,0,27.0,0,0,8.6625
4,896,3,1,22.0,1,1,12.2875


In [21]:
tree_clf.predict(tdf.drop(['PassengerId'], axis=1))

array([0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1,
       0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0,
       0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0,
       1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0,
       0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1,
       1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0,
       1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0,
       0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0,
       0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1,
       0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0,

In [22]:
result = pd.DataFrame()
result['PassengerId'] = np.array(tdf['PassengerId'])
result['Survived'] = tree_clf.predict(tdf.drop(['PassengerId'], axis=1))

In [23]:
result.to_csv('result.csv', index=False)